In [1]:
import psutil
total_memory = psutil.virtual_memory().total / 1e9  # Convert bytes to GB
print(f"Total memory: {total_memory:.2f} GB")

import os
num_cores = os.cpu_count()
print(f"Number of cores: {num_cores}")

Total memory: 17.18 GB
Number of cores: 8


In [2]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=4, threads_per_worker=2
)
client = Client(cluster)

print(client)
print(client.dashboard_link)

/Users/abinthomas/.pyenv/versions/3.12.3/lib/python3.12/contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.58s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


<Client: 'tcp://127.0.0.1:54410' processes=4 threads=8, memory=16.00 GiB>
http://127.0.0.1:8787/status


In [3]:
import pandas as pd
import dask.array as da
import dask.dataframe as dd
import logging

# Enable debug-level logging for Dask
# logging.basicConfig(level=logging.DEBUG)

In [4]:
%%time

"""Fit the BGM model on 10M rows of data"""

from data_transformer import DistributedDataTransformer

n_files = 200
num_partitions = 20

# Not worth doing this in dask
pd.concat([pd.read_csv("assets/Credit.csv")[["Amount"]]]*(n_files//num_partitions), ignore_index=True).to_csv("assets/Credit_large.csv", index=False)

train_data = dd.read_csv(["assets/Credit_large.csv" for _ in range(num_partitions)])[["Amount"]].repartition(npartitions=num_partitions)

transformer = DistributedDataTransformer(train_data)
transformer.fit()


Initialization 0


/Users/abinthomas/.pyenv/versions/3.12.3/envs/vgm/lib/python3.12/site-packages/dask/base.py:1103: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


  Iteration 10	 time lapse 69.92864s	 ll change 139955.00983
  Iteration 20	 time lapse 30.66324s	 ll change 54669.25238
  Iteration 30	 time lapse 32.52059s	 ll change 26047.94393
  Iteration 40	 time lapse 31.52468s	 ll change 15615.08877
  Iteration 50	 time lapse 32.84570s	 ll change 12189.85846
  Iteration 60	 time lapse 33.04623s	 ll change 7656.67696
  Iteration 70	 time lapse 35.66511s	 ll change 5621.18725
  Iteration 80	 time lapse 35.65871s	 ll change 5081.47197
  Iteration 90	 time lapse 37.03756s	 ll change 5847.71793
  Iteration 100	 time lapse 35.66207s	 ll change 10763.49993
Initialization did not converge. time lapse 374.55262s	 lower bound -42235135.65552.


/Users/abinthomas/Desktop/vgm/dbgm/dbgm.py:167: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


CPU times: user 4min 41s, sys: 17.1 s, total: 4min 58s
Wall time: 6min 19s


In [5]:
# %%time

# """Transform and inverse transform 10M rows of data"""

# transformed_train_data = transformer.transform(
#     train_data.to_dask_array(lengths=True)
# )
# inverse_transformed_train_data = transformer.inverse_transform(
#     transformed_train_data
# )
# regen_df = dd.from_dask_array(inverse_transformed_train_data, columns=["Amount"])

# print(dd.from_dask_array(da.abs(train_data.to_dask_array(lengths=True) - regen_df.to_dask_array(lengths=True)))[0].describe().compute())

In [6]:
%%time

"""Transform and inverse transform 1B rows of data"""

n_files = 20000
num_partitions = 2000

# Not worth doing this in dask
pd.concat([pd.read_csv("assets/Credit.csv")[["Amount"]]]*(n_files//num_partitions), ignore_index=True).to_csv("assets/Credit_large.csv", index=False)

train_data = dd.read_csv(["assets/Credit_large.csv" for _ in range(num_partitions)])[["Amount"]].repartition(npartitions=num_partitions)

transformed_train_data = transformer.transform(
    train_data.to_dask_array(lengths=True)
)
inverse_transformed_train_data = transformer.inverse_transform(
    transformed_train_data
)
regen_df = dd.from_dask_array(inverse_transformed_train_data, columns=["Amount"])

print(dd.from_dask_array(da.abs(train_data.to_dask_array(lengths=True) - regen_df.to_dask_array(lengths=True)))[0].describe().compute())

/Users/abinthomas/.pyenv/versions/3.12.3/envs/vgm/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 11.09 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


count    9.968400e+08
mean     3.389181e+00
std      1.245775e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.177351e+04
Name: 0, dtype: float64
CPU times: user 6min 15s, sys: 42.8 s, total: 6min 58s
Wall time: 10min 57s


In [7]:
client.close()
cluster.close()

2024-11-28 05:04:12,248 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/Users/abinthomas/.pyenv/versions/3.12.3/envs/vgm/lib/python3.12/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/abinthomas/.pyenv/versions/3.12.3/envs/vgm/lib/python3.12/site-packages/distributed/worker.py", line 1269, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/abinthomas/.pyenv/versions/3.12.3/envs/vgm/lib/python3.12/site-packages/distributed/utils_comm.py", line 441, in retry_operation
    return await retry(
           ^^^^^^^^^^^^
  File "/Users/abinthomas/.pye